In [146]:
#Comiezo importando librerias 
import matplotlib.pyplot as plt
import pandas as pd
from surprise.model_selection import train_test_split
from surprise import Dataset, Reader, SVD
%matplotlib inline

In [147]:
#Datasets
df_p = pd.read_csv('platform.csv')
df_r = pd.read_csv('rating_movies.csv')

In [148]:
df_p.sample(5)

,title,gender,movieId,platform
19301,the adventures of sharkboy and lavagirl,children & family movies,ns8184,netflix
8749,bell bottom,action,as8750,amazon
11114,night at the museum: battle of the smithsonian,"action-adventure, comedy, family",ds1447,disney
20507,capital one: college bowl,"game shows, reality, sports",hs583,hulu
12598,still game,"british tv shows, classic & cult tv, internati...",ns1481,netflix


In [149]:
df_r.sample(5)

,userId,score,timestamp,movieId
10629515,120490,4.0,2000-03-03,as953
3452270,35953,4.0,2017-07-05,ds257
809131,8290,4.0,2008-01-12,as4787
9636219,109607,2.0,2003-03-05,ns1570
6096526,62866,5.0,2017-07-14,as5889


In [150]:
df_p = df_p[['movieId', 'title']]

In [151]:
df_p = df_p.reset_index(drop=True)

In [152]:
N_filas = 100000 # We put a limit on the dataset

reader = Reader(rating_scale=(1, 5))

data = Dataset.load_from_df(df_r[['userId', 'movieId', 'score']][:N_filas], reader)



# Separating the data

trainset, testset = train_test_split(data, test_size=.25)

In [153]:
# Training the model SVD

model = SVD()
model.fit(trainset)

In [154]:
# Prediction

predictions = model.test(testset)

In [155]:
predictions[1]

Prediction(uid=242, iid='hs2276', r_ui=4.5, est=3.840253143785164, details={'was_impossible': False})

In [156]:
# Making a random prediction for user and movie

model.predict(35487,'ns8098')

Prediction(uid=35487, iid='ns8098', r_ui=None, est=3.595936544659656, details={'was_impossible': False})

In [157]:
'''
.A random user is chosen and I make a recommendation
.We take films that have been highly rated
'''

usuario = 35487; rating = 4   
df_user = df_r[(df_r.userId == usuario) & (df_r.score >= rating)]
df_user = df_user.reset_index(drop=True)

df_user = pd.merge(df_user, df_p[['movieId', 'title']], on='movieId', how='left')
df_user

,userId,score,timestamp,movieId,title
0,35487,4.0,1996-04-07,ns7682,otherlife
1,35487,4.0,1996-04-07,ns3147,scooby-doo!: mystery incorporated
2,35487,5.0,1996-04-07,ds1017,life is ruff
3,35487,5.0,1996-04-07,ns900,illegal woman
4,35487,4.0,1996-04-07,as7715,raymond & miguel
5,35487,5.0,1996-04-07,ns2340,prem ratan dhan payo
6,35487,4.0,1996-04-07,as5262,nishabdham
7,35487,5.0,1996-04-07,ns2851,the last bomb of the second world war
8,35487,5.0,1996-04-07,ns1169,dota: dragon's blood
9,35487,4.0,1996-04-07,as8428,rifftrax: contamination


In [158]:
u_recommendations = df_p.iloc[:4499].copy()
print(u_recommendations.shape)
u_recommendations.head()

(4499, 2)


,movieId,title
0,as1,the grand seduction
1,as2,take care good night
2,as3,secrets of deception
3,as4,pink: staying true
4,as5,monster maker


In [159]:
#Movies that have already been seen are extracted

usuario_vistas = df_r[df_r['userId'] == usuario]
print(usuario_vistas.shape)

usuario_vistas.head()

(26, 4)


,userId,score,timestamp,movieId
3408964,35487,3.0,1996-04-07,ns8098
3408965,35487,4.0,1996-04-07,ns7682
3408966,35487,4.0,1996-04-07,ns3147
3408967,35487,5.0,1996-04-07,ds1017
3408968,35487,5.0,1996-04-07,ns900


In [160]:
# Removing the movies has already seen for the user

u_recommendations = u_recommendations[~u_recommendations['movieId'].isin(usuario_vistas['movieId'])]

u_recommendations.shape

(4497, 2)

In [161]:
# Recommendation -->

u_recommendations['Estimate_Score'] = u_recommendations['movieId'].apply(lambda x: model.predict(usuario, x).est)

In [162]:
u_recommendations = u_recommendations.sort_values('Estimate_Score', ascending=False)
print(u_recommendations.head(10))

     movieId                                      title  Estimate_Score
118    as119             where is chicky? - chapter two        4.003503
1853  as1854                         lincoln@gettysburg        3.932304
1069  as1070            secret life of the hospital bed        3.901062
2467  as2468  fireman sam: the great fire of pontypandy        3.900741
4156  as4157                                      darra        3.899823
3794  as3795                             painted horses        3.888790
2573  as2574                           eic vs the world        3.875554
1601  as1602       morphle and mila dinosaur adventures        3.874287
2679  as2680                 detectives: my killer case        3.866570
2736  as2737                                     dakota        3.865806


__Prediccion con un usuario o pelicula__

In [163]:
def movie_recommendation(userId, movieId):
    # We obtain the prediction of the rating that the user would give to the movie
    prediction = model.predict(userId, str(movieId))

    # If the prediction is greater than or equal to 3.5, the movie is recommended
    if prediction.est >= 3.5:
        return "Recommended", prediction.est
    else:
        return "Not recommended", prediction.est

In [164]:
#Function testing
movie_recommendation(59, 'ns65')

('Not recommended', 3.2694139554041595)

 __Model evaluation__

In [165]:

from surprise import accuracy

accuracy.rmse(predictions)

RMSE: 0.9609


0.9608552151975971

In [166]:
#Optimizo hiperparametros
from surprise.model_selection import cross_validate
import numpy as np

rmse_test_means = []
factores = [1,2,4,8,16,32,64,128]

for factor in factores:
    print(factor)
    model = SVD(n_factors=factor)
    cv = cross_validate(model, data, measures=['RMSE'], cv = 3, verbose=True)
    rmse_test_means.append(np.mean(cv['test_rmse']))

1
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9474  0.9494  0.9531  0.9499  0.0024  
Fit time          0.48    0.44    0.47    0.46    0.02    
Test time         0.48    0.19    0.23    0.30    0.13    
2
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9475  0.9473  0.9537  0.9495  0.0030  
Fit time          0.42    0.51    0.48    0.47    0.04    
Test time         0.20    0.44    0.21    0.28    0.11    
4
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9447  0.9434  0.9587  0.9490  0.0069  
Fit time          0.44    0.49    0.48    0.47    0.02    
Test time         0.20    0.21    0.43    0.28    0.10    
8
Evaluating RMSE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9510  0.9

In [167]:
#Creation function to extract title

def title_extraction(movieId):
    return df_p[df_p.movieId == movieId].title.iloc[0].title()

In [168]:
#Function test
title_extraction("ds568")

'A Tale Of Two Critters'

In [174]:
#fusion process
def title_recommendation(userId, movieId):
    recomendado, puntaje = movie_recommendation(userId, movieId)
    titulo_1 = title_extraction(movieId)
    return (recomendado, puntaje,titulo_1)

In [176]:
#Test
title_recommendation(999,"ds1017")

('Recommended', 3.54167687205084, 'Life Is Ruff')

In [177]:

import gradio as gr

title = str("Recommendation system")

with gr.Blocks(title=title) as demo:
    user_number = gr.inputs.Number(label='Nickname')
    movie = gr.Textbox(label='Number Movie')
    recommendation = gr.Button('Search recommendations')
    title = gr.Textbox(label='Recommendation name')
    output = gr.Textbox(label= 'Qualification')
    score = gr.Textbox(label='Qualification percentage')
    recommendation.click(fn = title_recommendation, inputs=[user_number,movie], outputs=[output, score,title])
demo.launch(share = True)

c:\Users\matia\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\inputs.py:59: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
c:\Users\matia\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)


Running on local URL:  http://127.0.0.1:7867
Running on public URL: https://4ae44deb3082af58fa.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
